# Phd-teach-PhD: Approaching science with AI

**An introduction to scientific machine learning**

Guilherme Zagatti      
PhD candidate NUS NGS/IDS   
<gzagatti@u.nus.edu>   

## Outline

1. Motivation: Dynamical systems describe changes we observe

2. Julia

    Intro to the language and ecosystem
    
3. A formal introduction to ODEs and SciML

    The initial value problem
    
    `DifferentialEquations.jl`
    
    Fitting data to ODEs, classical approaches
    
4. NeuralODE

    The initial value problem reviewed
    
    `Flux.jl`

5. Exercises

## Motivation

**Dynamical systems describe changes we observe**

## "It is then by cause that we define time" (Poincaré 1903)

How do we learn about cause and effect if not by how we observe things evolving? 

**Dynamical systems** are an evolution rule that defines a trajectory as a map ([Meiss 2007](https://sinews.siam.org/Details-Page/differential-dynamical-systems-revised-edition-4)):

<br/>

$$
    \text{time} ↦ \text{set of states}
$$

<br/>

As a mathematics discipline, the study of dynamical systems originates at the end of 19th century spearheaded by [Henri Poincaré](https://plato.stanford.edu/entries/poincare/) who also developed an extensive theory of sciences.

### Physics: coupled oscillator

Describes the movement of an object with mass $m$ attached to a spring under force $F$.

<div style="column-count: 2; column-rule: 1px dotted #e0e0e0; display: flex;">
<div style="display: flex; width: 300px;">
<img src="../assets/mass-spring.png" style="display: block; margin:auto;"></img>
</div>
<div style="display: flex; align-items: center;">
$$
F = ma \Rightarrow
\frac{d^2 x}{dt^2} =-\frac{kx}{m}
$$
</div>
</div>

### Ecology: Lotka-Volterra

Describes the dynamics of two-competing species, the prey $x$ and predator $y$.

<div style="column-count: 2; column-gap: 5px; display: flex;">
<div style="display: flex; width: 300px;">
<img src="../assets/fox-rabbit.png" style="display: block; margin:auto;"></img>
</div>
<div style="display: flex; align-items: center;">
$$
\begin{aligned}
\frac{dx}{dt} &= \alpha x - \beta xy \\
\frac{dy}{dt} & = \delta xy - \gamma y
\end{aligned}
$$
</div>
</div>

### Epidemiology: SIR

Susceptible-infected-recovered. Describes the evolution of an infectious disease which is transmitted with rate $\beta$ when a susceptible and an infected individual come into contact and disappears with rate $\gamma$.

<div style="column-count: 2; column-gap: 5px; display: flex;">
<div style="display: flex; width: 300px;">
<img src="../assets/sneeze.png" style="display: block; margin:auto;"></img>
</div>
<div style="display: flex; align-items: center;">
$$
\begin{aligned}
\frac{ds}{dt} &= - \beta si \\
\frac{di}{dt} & = \beta si - \gamma i \\
\frac{dr}{dt} &= \gamma i
\end{aligned}
$$
</div>
</div>

### Economics: Solow growth model

The production function $f(.)$ describes growth as a function of $k$. The rate of capital growth is constrained by the savings rate $s$ and the depreciation rate of capital $\delta$. 

<div style="column-count: 2; column-gap: 5px; display: flex;">
<div style="display: flex; width: 300px;">
<img src="../assets/growth.png" style="display: block; margin:auto;"></img>
</div>
<div style="display: flex; align-items: center;">
$$
\frac{dk}{dt} = s f(k) - \delta k
$$
</div>
</div>

### Describing nature with models

A lot ingenuity and observation is required to develop models that are both **consistent with reality and existing theory**.

Science is a combination of **marginal revolutions and paradigm shifts**.

Scientific machine learning is a **set of tools for automated model discovery** to support the development of models from data.
    
<img src="../assets/khun.png" style="display: block; margin:auto; width: 300px;"></img>

"*It is not sufficient for each elementary phenomenon to obey simple laws, all those to be combined must obey the same law as well.* "<br/>Poincaré 1902

## The <img src="../assets/julia-logo.svg" style="display: inline; vertical-align: text-top; width: 36pt; "></img> programming language

**Fast** designed for high-performance, JIT compiled code.

**Reproducible environment** `Manifest.toml` contains all the instructions to reproduce the environment; like the one in this notebook.

**General** from data-wrangling through data-analytics to data-reporting

**Dynamic** feels like Python

**Composable** multiple dispatch as a paradigm and functional programming, not quite like Python

**Open source** MIT license and active developer community in the scientific machine learning field.

Visit [Julia's](https://julialang.org) website to get started. 

## Installation



To follow the course, make sure to install the latest stable version of `Julia` from its [official page](https://julialang.org/downloads/). 

Then, clone this repository:

```
> git clone git@github.com:gzagatti/phdteachphd-sciml.git
```

After downloading the repository, activate the Julia environment and install all the dependencies:

```
> cd phdteachphd-sciml/
> julia
               _
   _       _ _(_)_     |  Documentation: https://docs.julialang.org
  (_)     | (_) (_)    |
   _ _   _| |_  __ _   |  Type "?" for help, "]?" for Pkg help.
  | | | | | | |/ _` |  |
  | | |_| | | | (_| |  |  Version 1.6.1 (2021-04-23)
 _/ |\__'_|_|_|\__'_|  |  Official https://julialang.org/ release
|__/                   |

julia> ]
(@v1.6) pkg> activate .
  Activating environment at `./phdteachphd-sciml/Project.toml`
(@v1.6) pkg> instantiate
  Resolving package versions...
  ...
```
